# Méthodes numériques appliquées à l'analyse littéraire

**Objectif**: Etablir ensemble un protocole de recherche outillé. 

 - Comment faire pour répondre à la question?
 - Quelles données allons nous stocker et interroger? 
 - Quelles méthodes nous allons employer?
 - Quelles types d'analyse allons nous mobiliser? 

Nous avons volontairement choisi de nous concentrer sur des ressources webs.
Deux sites internets français qui sont des réseaux sociaux 
et qui nous paraissent avoir des éléments en mesure de répondre à la question

- babelio.com
- senscritique.fr/livres


## ETAPE 1: Collecter les métadonnées nécessaires 
## WebMining

1. Requeter la page web > Récupérer le texte programmatiquement
> La machine va faire la même opération que lorsqu'on se connecte depuis un navigateur à l'adresse indiquée
> une requete client qui renvoie une réponse du serveur (un code et un contenu textuel le HTML)


![title](./page_web.png)

In [17]:
import requests
URL = "https://www.babelio.com/decouvriretiquettes.php"
response = requests.get(URL)
if response.status_code == 200:
    page = response.text
    print(page[:2000])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN" "https://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd"><html version="XHTML+RDFa 1.0" xmlns="https://www.w3.org/1999/xhtml" xml:lang="fr"><head><meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1" /><meta http-equiv="Content-language" content="fr-FR" /><meta http-equiv="cache-control" content="no-cache"><meta http-equiv="pragma" content="no-cache"><meta name="tdm-reservation" content="1"><meta http-equiv="expires" content="-1"><link rel="icon" href="/favicon192x192.png" /><meta charset="iso-8859-1"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="user-scalable=no, width=device-width, initial-scale=1, maximum-scale=1"><meta property="og:type" content="website"></meta><title>Babelio - Découvrez des livres, critiques, extraits, résumés</title><meta name="title" content="Babelio - Découvrez des livres, critiques, extraits, résumés" /><meta name="description" content="Le site où les passio

Pour avoir plus de détails sur le fonctionnement du module requests. Rendez vous sur la page de la [documentation de Requests](https://requests.readthedocs.io/en/latest/)

### HTML PARSING
Il faut ensuite convertir le texte à un arbre car le HTML est un langage semistructuré en arbre. Vous connaissez déjà le XML donc vous savez. 
On va utiliser un module pour parcourir le texte et le convertir pour pouvoir découper ce qu'on veut extraire comme information en utilisant les balises HTML et le CSS pour identifier les morceaux qu'on veut collecter.

> La machine va reproduire ce que l'utilisateur fait manuellement sur une page web.


![identifier la balise](identify_tag.png)

In [18]:
from bs4 import BeautifulSoup as bs

html_tree = bs(page, "lxml")
all_links = html_tree.find_all("a")
print(all_links[:10])

[<a href="/actualites.php" id="bleb4i" rel="nofollow">Accueil</a>, <a href="/mabibliotheque.php" rel="nofollow">Mes livres</a>, <a href="/ajoutlivres.php" rel="nofollow">Ajouter des livres</a>, <a href="/decouvrir.php">Livres</a>, <a href="/decouvrirauteurs.php">Auteurs</a>, <a href="/decouvrirmembres_v2.php?s=1">Lecteurs</a>, <a href="/dernierescritiques.php">Critiques</a>, <a href="/dernierescitations.php">Citations</a>, <a href="/listes-de-livres/">Listes</a>, <a href="/quiz/">Quiz</a>]


Pour savoir comment extraire certaines balises et le texte interne à la balise. RDV sur la page de [documentation de BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [ ]:
On veut récupérer tous les pages d'ouvrages qui ont cette étiquette.
On a remarqué que toutes les étiquettes ont un motif récurent 


## A vous de jouer!
* Identifier les pages que vous voulez extraire
* Ouvrez les dans votre navigateur
* Activer la console développeur (Ctrl+Maj+I ou Cmd + Maj +J)
* Identifier les balises qui contiennent les informations que vous voulez stocker et notez-les


In [19]:
def get_the_soup(url):
    '''Recupérer la page HTML à partir d'une url et transformer le HTML en un arbre à parcourir'''
    r = requests.get(url) 
    #proxies=proxy)
    if r.status_code == 200:
        return bs(r.text, "lxml")

In [20]:
# On l'utilise comme ceci
# on dit qu'on instancie la fonction ( on appelle la fonction avec un paramêtre et elle s'execute)
# page est une variable et get_the_soup est une fonction qui prend en paramètre une url 
#et retourne du HTML que l'ont peut interroger
page = get_the_soup("https://www.babelio.com/decouvriretiquettes.php")
# la variable page contient tout l'arbre HTML, on va ensuite ecrire une fonction 
# qui permet d'extraire les métadonnées souhaitées


Pour des raisons de réutilisation et adaptation,
voici une fonction d'exemple qui prend en parametre la page HTML préalablement parsée 
et qui renvoie le titre de la page 

In [21]:
def extract_title(page):
    '''A partir d'une page parsée, 
    découper la valeur qui nous intéresse ici en nous aidant des balises HTML et du CSS
    et le renvoyer pour le stocker'''
    title = page.find("h1", {"class": "titre"})
    return title.text

Voici une autre fonction qui renvoie une liste de liens.

In [22]:
def get_links(url, page):
    links = []
    for a in page.find_all("a"):
        links.append(a.get("href"))
    return links

# elle peut aussi s'ecrire de manière plus compacte
def get_links(page):
    return [a.get("href") for a in page.find_all("a")]
# testons donc en chainant les fonctions
page_etiquettes = get_the_soup("https://www.babelio.com/decouvriretiquettes.php") 
liens_etiquettes = get_links(page_etiquettes)
print(liens_etiquettes)

['/actualites.php', '/mabibliotheque.php', '/ajoutlivres.php', '/decouvrir.php', '/decouvrirauteurs.php', '/decouvrirmembres_v2.php?s=1', '/dernierescritiques.php', '/dernierescitations.php', '/listes-de-livres/', '/quiz/', '/groupes', '/questions', '/prix-babelio', '/article/1939/Retrouvez-toutes-nos-rencontres-dauteurs-et-inscr', 'https://www.laboutiquebabelio.com/', '/', '/decouvrir.php', '/decouvrirauteurs.php', '/decouvrirmembres_v2.php?s=1', '/dernierescritiques.php', '/dernierescitations.php', '/listes-de-livres/', '/quiz/', '/groupes', '/questions', '/prix-babelio', '/article/1939/Retrouvez-toutes-nos-rencontres-dauteurs-et-inscr', 'https://www.laboutiquebabelio.com/', '/decouvrir.php', '/actualites.php', '/mabibliotheque.php', '/connection.php', '/connection.php', '/decouvrir.php', '/decouvrir.php', '/a_paraitre.php', '/decouvrirn.php', '/decouvrirauteurs.php', '/dernierescritiques.php', '/dernierescitations.php', '/derniersvideos.php', '/decouvriretiquettes.php?p=1', '/decouv

On remarque plusieurs choses: on a des urls en double, on a des liens relatifs et des liens absolus et on a potentiellement des références à la même page. Pour pouvoir extraire toutes les pages de chaque étidette on a besoin d'une url absolue soit une adresse complète on va donc améliorer la fonction et enchainer dans la fonction

In [16]:
import os
#BASE_URL est a changer selon le site sur lequel vous travaillez
BASE_URL = "http://babelio.fr"
def get_links(url):
    page = get_the_soup(url)
    links = []
    for a in page.find_all("a"):
        if not a.get("href").startswith("http"):
            link = os.path.join(BASE_URL, a.get("href"))
            if link != BASE_URL+"/" and link != url:
                links.append(link)
    # ici on dédoublonne les liens
    return list(set(links))

## ETAPE 2: Stocker les informations
## Data Management

Pour pouvoir enquêter, il faut stocker et lire les informations. A partir d'un certain seuil de taille, on utilise une base de données pour enregistrer et lire les données.
A notre niveau, on va simplement stocker dans un fichier tabulaire CSV pour les enregistrer et les lire. 


In [24]:
import csv
def write_data(filename, data):
    with open(filename, "w") as f:
        csv_writer = csv.dictWriter(f, filednames = data.keys())
        csv_writer.writeheader()
        for row in data:
            csv_writer.writerow(row)
    return
def read_data(filename):
    with open(filename, "r") as f:
        csv_reader = csv.dictReader(f)
        return [row for row in csv_reader]

# ou alternativement
import pandas as pd
def read_dataframe(filename):
    return pd.read_csv(filename)

        
        

In [ ]:
with open(f"TAG-{row['name']}-details.csv", "w") as fd:
    detail_writer = csv.DictWriter(fd, fieldnames=["id","auteur","titre","resume","rating","votes","linked_books","tags"])
    detail_writer.writeheader()
    #["id", "Auteur", "Titre", "Resumé", "Note générale", "Total des Notes", "ID des livres conseillés", "Etiquettes associées"]
    with open(f"TAG-{row['name']}-booklist.csv", "w") as f:
        list_writer = csv.writer(f, delimiter=",")
        list_writer.writerow("id,url\n")
        for page in pages:
            for n in page.find_all("a"):
                href = n.get("href")
                id = href.split("/")[-1]
                if href is not None and href.startswith("/livres/") and  "#" not in href and id.isnumeric():
                    url = BASE_URL+ href
                    print(url)
                    id = int(id)
                    list_writer.writerow(f"{id},'{url}'\n")
                    book_page = get_the_soup(url)
                    book_details = get_book_detail(book_page, id)
                    detail_writer.writerow(book_details)


Maintenant on a de quoi enquêter, on va pouvoir faire les vérifications d'usage pour s'assurer de la cohérence de notre corpus.
